# Análisis del riesgo de incumplimiento de los prestatarios

En este informe se mostrarán los estudios realizados para determinar si el estado civil, el número de hijos, el nivel de ingresos y el propósito del préstamo, tienen incidencia en el cumplimiento del pago del crédito.
Para ello, se revisará la data entregada, se corregirán los datos necesarios, se revisará si hay registros duplicados. Todo esto, estará junto con la explicación de cada paso y decisión tomada. Posteriormente, se mostrará la clasificación de los datos y se generará información para la división de préstamos del banco.
Las preguntas a responder son:

-¿Existe una correlación entre tener hijos y pagar a tiempo?

-¿Existe una correlación entre la situación familiar y el pago a tiempo?

-¿Existe una correlación entre el nivel de ingresos y el pago a tiempo?

-¿Cómo afecta el propósito del crédito a la tasa de incumplimiento?

## Se abre el archivo de datos y mira la información general. 
Primero se cargan todas las librerías y se lee el archivo csv a estudiar.

In [1]:
# Se cargan las librerías a utilizar en este informe
import pandas as pd
import numpy as np

In [2]:
# Se cargan los datos
try:
    data = pd.read_csv('credit_scoring_eng.csv')
except:
    data = pd.read_csv('/datasets/credit_scoring_eng.csv')

## Exploración de datos

**Descripción de los datos**
- `children` - el número de hijos en la familia
- `days_employed` - experiencia laboral en días
- `dob_years` - la edad del cliente en años
- `education` - la educación del cliente
- `education_id` - identificador de educación
- `family_status` - estado civil
- `family_status_id` - identificador de estado civil
- `gender` - género del cliente
- `income_type` - tipo de empleo
- `debt` - ¿había alguna deuda en el pago de un préstamo?
- `total_income` - ingreso mensual
- `purpose` - el propósito de obtener un préstamo

Se revisa el conjunto de datos, para saber cuántas columnas y filas tiene, ver si tienen valores ausentes para ir analizando posibles problemas.

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
# se verán las primeras 10 filas
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


En los datos no hay ID para cada cliente, por lo que al momento de revisar si hay data duplicada, esto podría generar dudas y se tendrán que evaluar los casos, si es que existieran filas duplicadas.

Para la columna `education` es recomendable escribir todo en minúsculas, para revisar la data.

Para la columna `purpose` se revisarán los datos únicos, por si se tuvieran que homologar algunos motivos, por ejemplo, 'to have a wedding', 'having a wedding' y 'wedding ceremony' se podrían agrupar como 'wedding'

In [5]:
print(data['purpose'].unique())

['purchase of the house' 'car purchase' 'supplementary education'
 'to have a wedding' 'housing transactions' 'education' 'having a wedding'
 'purchase of the house for my family' 'buy real estate'
 'buy commercial real estate' 'buy residential real estate'
 'construction of own property' 'property' 'building a property'
 'buying a second-hand car' 'buying my own car'
 'transactions with commercial real estate' 'building a real estate'
 'housing' 'transactions with my real estate' 'cars' 'to become educated'
 'second-hand car purchase' 'getting an education' 'car'
 'wedding ceremony' 'to get a supplementary education'
 'purchase of my own house' 'real estate transactions'
 'getting higher education' 'to own a car' 'purchase of a car'
 'profile education' 'university education'
 'buying property for renting out' 'to buy a car' 'housing renovation'
 'going to university']


Para la columna `days_employed` se detectó lo siguiente:
- Hay cantidad de días "grandes", por ejemplo, el máximo de días registrados es '401755.40047533', se debe analizar si en vez de días se ingresaron horas trabajadas y decidir qué hacer con los datos.

In [6]:
print(data['days_employed'].max())

401755.40047533


- Es de tipo float64, al ser días tal vez sería mejor considerar el dato como int64, es decir, como número entero. Esto se tiene que estudiar, de acuerdo a lo mencionado en punto anterior.
- Hay datos con números negativos, por lo que se debe revisar y decidir qué hacer con estos.
- Tiene valores ausentes, esto se puede deber a que la persona no ha trabajado. Se deben analizar los datos.

La columna `total_income` también tiene valores ausentes, coincide el número de data con la columna con `days_employed`, lo que podría reafirmar que esos clientes no han trabajado, por lo tanto, no han generado ingresos.

In [7]:
#Se busca qué columnas tienen valores ausentes
print(data.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


Se puede ver que existen 2.174 registros NaN para las columnas `days_employed` y `total_income`

Se verificará que estos valores ausentes se encuentren en las mismas filas.

In [8]:
# Se filtran las filas con valores NaN en las columnas "days_employed" y "total_income" al mismo tiempo
filas_con_nans = data[data[['days_employed', 'total_income']].isna().all(axis=1)]
# Se obtiene la cantidad de filas que cumplen la condición
cantidad_filas_con_nans = len(filas_con_nans)
# Se muestra el resultado
print("Cantidad de filas con valores NaN en las columnas 'days_employed' y 'total_income':", cantidad_filas_con_nans)

Cantidad de filas con valores NaN en las columnas 'days_employed' y 'total_income': 2174


Por lo revisado, se encuentra que los 2.174 registros de valores ausentes se encuentran en las mismas filas, por lo que se podría interpretar como que esos clientes no han trabajado y es por eso, que no han generado ingresos. De todas maneras, se revisará la columna `income_type` para estos casos.

In [9]:
# Se muestra el contenido de la columna 'income_type' de las filas filtradas
contenido_columna_income_type = filas_con_nans['income_type']
print("El contenido de la columna 'income_type' en las filas con valores NaN en las columnas 'days_employed' y 'total_income' es:")
print(contenido_columna_income_type.unique())

El contenido de la columna 'income_type' en las filas con valores NaN en las columnas 'days_employed' y 'total_income' es:
['retiree' 'civil servant' 'business' 'employee' 'entrepreneur']


**Conclusión intermedia**

De acuerdo a lo revisado en la última consulta, las filas donde existen valores ausentes, corresponden a personas que tienen los siguientes tipos de ingreso (`income_type`):
'jubilado' 'funcionario' 'empresa' 'empleado' 'empresario'

Por lo tanto, se desecha la idea de que las personas no han trabajado. Para evaluar si se completarán los valores ausentes, a continuación, se calculará el porcentaje de los valores ausentes en comparación con el conjunto de datos completo:

In [10]:
porcentaje = cantidad_filas_con_nans / len(data)
print(f'El porcentaje de valores ausentes es de un {porcentaje:.2%}')

El porcentaje de valores ausentes es de un 10.10%


Se revisará el resto de columnas para los 2.174 registros que tienen valores ausentes en las columnas `days_employed` y `total_income`. Esto para determinar si podría existir algún patrón para tener estos datos ausentes.

Para esto primero, se revisará la columna `dob_years`:

In [11]:
contenido_columna_dob_years = filas_con_nans['dob_years']
print("El contenido de la columna 'dob_years' en las filas con valores NaN en las columnas 'days_employed' y 'total_income' es:")
print(contenido_columna_dob_years.sort_values().unique())
print('')

El contenido de la columna 'dob_years' en las filas con valores NaN en las columnas 'days_employed' y 'total_income' es:
[ 0 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41
 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65
 66 67 68 69 70 71 72 73]



A continuación, se revisan las distribuciones utilizando value_counts(normalize=True). De esta manera, se obtiene el comportamiento porcentual de cada elemento dentro de ambas variables (`days_employed` y `total_income`).

In [12]:
# Comprobación de la distribución
# Primero se filtran las filas que tienen valores NaN en las columnas 'days_employed' y 'total_income' al mismo tiempo
filas_con_nans = data[data[['days_employed', 'total_income']].isna().all(axis=1)]

In [13]:
#Ahora se analizan las columnas restantes
columnas_a_analizar = ['children', 'dob_years', 'education', 'education_id', 'family_status',
                        'family_status_id', 'gender', 'income_type', 'debt', 'purpose']

for columna in columnas_a_analizar:
    distribucion_porcentual = filas_con_nans[columna].value_counts(normalize=True)
    print("Distribución porcentual de '{}' en las filas con valores NaN en 'days_employed' y 'total_income':".format(columna))
    print(distribucion_porcentual)


Distribución porcentual de 'children' en las filas con valores NaN en 'days_employed' y 'total_income':
children
 0     0.661914
 1     0.218491
 2     0.093836
 3     0.016559
 20    0.004140
 4     0.003220
-1     0.001380
 5     0.000460
Name: proportion, dtype: float64
Distribución porcentual de 'dob_years' en las filas con valores NaN en 'days_employed' y 'total_income':
dob_years
34    0.031739
40    0.030359
42    0.029899
31    0.029899
35    0.029439
36    0.028979
47    0.027139
41    0.027139
30    0.026679
28    0.026219
58    0.025759
57    0.025759
54    0.025299
56    0.024839
38    0.024839
52    0.024379
37    0.024379
33    0.023459
39    0.023459
50    0.023459
43    0.022999
45    0.022999
49    0.022999
51    0.022999
29    0.022999
46    0.022079
55    0.022079
48    0.021159
44    0.020239
53    0.020239
60    0.017939
62    0.017479
61    0.017479
32    0.017019
64    0.017019
23    0.016559
27    0.016559
26    0.016099
59    0.015639
63    0.013339
25    0.010

**Posibles razones por las que hay valores ausentes en los datos**

Al revisar los resultados, no se encuentra alguna relación para pensar en que se formó un patrón para completar estos datos vacíos.
En un principio, se podría haber pensado en la edad (`dob_years`) para tener un patrón, pero los valores mínimos son 0 y 19 años:
- Cero años, debe ser un error de ingreso de datos.
- 19 años, ya es mayor de edad por lo que podría trabajar.

Luego, se revisan las distribuciones arrojando los siguientes resultados:

-`children`: La mayoría de los registros con valores ausentes en `days_employed` y `total_income` tienen 0 ó 1 hijo. La presencia de valores como 20 hijos o un valor negativo (-1) es poco común en estas filas.

-`
dob_year`s: Hay una distribución más o menos uniforme de edades, aunque se observa una concentración en edades alrededor de 30-40 años

-`education` y `education_id`: Las personas con educación secundaria y `education_id` = 1 son las más comunes en estas filas. Las categorías en mayúsculas y en minúsculas pueden estar duplicadas o con diferentes formatos de texto, lo que debe ser corregido para analizar los datos.

-`family_status` y `family_status_id`: Las personas casadas y con un `family_status_id` = 0 son las más comunes en estas filas.

-`gender`: La mayoría de las personas en estas filas con valores ausentes son mujeres (F).

-`income_type`: Los empleados (employee) y personas con negocios (business) son los más comunes en estas filas.

-`debt`: La gran mayoría de las filas con valores ausentes no tienen deudas (`debt` = 0), mientras que una pequeña proporción tiene deudas (`debt` = 1).

-`purpose`: Hay una variedad de propósitos diferentes para préstamos en estas filas con valores ausentes. No se destaca un propósito específico..


Por todlo mencionadoto, no se encuentra un patrón para los valores vacíos y al ser un 10.1% de la data, se revisará si se deben completar los valores ausentes. Para esto se debe revisar si hay valores atípicos significativos.

**Conclusiones**

Finalmente, no se encontró un patrón para determinar el comportamiento de valores ausentes. 

Como se mencionó al principio, se deben revisar las diferentes columnas, a continuación se mencionarán los diferentes casos:

- Para la columna `children`, se debe revisar los datos, ya que existe al menos una fila donde se registró con número negativo.
- Para la columna `education` es recomendable escribir todo en minúsculas, para revisar la data.
- Para la columna `purpose` se revisarán los datos únicos, por si se tuvieran que homologar algunos motivos, por ejemplo, 'to have a wedding', 'having a wedding' y 'wedding ceremony' se podrían agrupar como 'wedding'
- Para la columna `days_employed` se revisará lo siguiente:
    - Hay cantidad de días "grandes", por ejemplo, el máximo de días registrados es '401755.40047533', se debe analizar si en vez de días se ingresaron horas trabajadas y decidir qué hacer con los datos.
    - Es de tipo float64, al ser días tal vez sería mejor considerar el dato como int64, es decir, como número entero. Esto se tiene que estudiar, de acuerdo a lo mencionado en punto anterior.
    - Hay datos con números negativos, por lo que se debe revisar y decidir qué hacer con estos.
    - Por último luego de estas correcciones decidir si se completan los valores ausentes y con qué parámetro.

## Transformación de datos

Se revisarán las diferentes columnas para detectar si hay problemas a corregir.

In [14]:
# Se revisa contenido de campo `education`
print(data['education'].unique())

["bachelor's degree" 'secondary education' 'Secondary Education'
 'SECONDARY EDUCATION' "BACHELOR'S DEGREE" 'some college'
 'primary education' "Bachelor's Degree" 'SOME COLLEGE' 'Some College'
 'PRIMARY EDUCATION' 'Primary Education' 'Graduate Degree'
 'GRADUATE DEGREE' 'graduate degree']


In [15]:
# Se forzará a que todo esté con letras minúsculas
data['education'] = data['education'].str.lower()

In [16]:
# Se revisa nuevamente contenido de campo `education`
print(data['education'].unique())

["bachelor's degree" 'secondary education' 'some college'
 'primary education' 'graduate degree']


Ahora se puede trabajar con la columna.
A continuación se verificarán los datos de la columna `children`

In [17]:
# Veamos la distribución de los valores en la columna `children`
data['children'].describe()

count    21525.000000
mean         0.538908
std          1.381587
min         -1.000000
25%          0.000000
50%          0.000000
75%          1.000000
max         20.000000
Name: children, dtype: float64

In [18]:
#Se verán los diferentes números de hijos registrados en la data
print(data['children'].sort_values().unique())

[-1  0  1  2  3  4  5 20]


In [19]:
#Se verificará la cantidad de datos de hijos con valor -1 y 20
hijos_nro_negativo = (data['children'] == -1)
hijos_nro_veinte = (data['children'] == 20)
print('La cantidad de registros que indican -1 hijos es:',hijos_nro_negativo.sum())
print('\nLa cantidad de registros que indican 20 hijos es:',hijos_nro_veinte.sum())

La cantidad de registros que indican -1 hijos es: 47

La cantidad de registros que indican 20 hijos es: 76


In [20]:
#Se calculará el porcentaje de estos datos
porcentaje = hijos_nro_negativo.sum() / len(data['children'])
print(f'El porcentaje de registros que indican -1 hijos es: {porcentaje:.2%}')

porcentaje = hijos_nro_veinte.sum() / len(data['children'])
print(f'El porcentaje de registros que indican 20 hijos es: {porcentaje:.2%}')

El porcentaje de registros que indican -1 hijos es: 0.22%
El porcentaje de registros que indican 20 hijos es: 0.35%


La columna `children` tiene registros con número negativo (-1) y tiene un valor máximo de 20 hijos. Esto corresponde al 0.22% y 0.35% respectivamente. Es posible que en ambos casos sea un error de escritura, por lo que se corregirá y se reemplazarán los valores -1 por 1 y los valores 20, por 2.

In [21]:
# reemplazar datos -1 por valor 1
data['children'] = data['children'].replace(-1,1)
# reemplazar datos 20 por valor 2
data['children'] = data['children'].replace(20,2)

In [22]:
# Comprobar la columna `children` de nuevo para asegurarnos de que todo está arreglado
#Se verán los diferentes números de hijos registrados en la data que fue corregida
print(data['children'].sort_values().unique())

[0 1 2 3 4 5]


La columna `children` fue corregida. De manera adicional, se creará el campo `Tiene_hijos`

In [23]:
#función para completar con 1 si tiene hijos, con 0 si no tiene hijos
def tiene_hijos(children):
    if children >0:
        return 1
    else:
        return 0

In [24]:
#asignación en nueva columna 'tiene_hijos'
data['tiene_hijos'] = data['children'].apply(tiene_hijos)

Para continuar el análisis, tal y como se mencionó anteriormente, para la columna `days_employed` se revisará lo siguiente:
- Hay datos con números negativos, por lo que se debe revisar y decidir qué hacer con estos.
- Hay cantidad de días "grandes", por ejemplo, el máximo de días registrados es '401755.40047533', se debe analizar si en vez de días se ingresaron horas trabajadas y decidir qué hacer con los datos.
- Es de tipo float64, al ser días tal vez sería mejor considerar el dato como int64, es decir, como número entero. Esto se tiene que estudiar, de acuerdo a lo mencionado en punto anterior.
- Por último luego de estas correcciones decidir si se completan los valores ausentes y con qué parámetro.


In [25]:
# Se revisa resumen de columna `days_employed`
data['days_employed'].describe()

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

In [26]:
#Se calculará cantidad de días negativos registrados en columna 'days_employed'
days_employed_negative = (data['days_employed'] < 0)
days_employed_negative.sum()

15906

In [27]:
#Se calculará el porcentaje de días negativos
porcentaje = days_employed_negative.sum() / len(data['days_employed'])
print(f'El porcentaje de registros con número negativo es: {porcentaje:.2%}')

El porcentaje de registros con número negativo es: 73.90%


Existe un 73.9% de data con número negativo, por lo que se sugiere aplicar el valor absoluto, ya que no podemos borrar las filas.

In [28]:
# Aplicando valor absoluto
data['days_employed'] = data['days_employed'].abs()

In [29]:
# Para comprobar lo realizado, se revisa resumen de columna nuevamente
data['days_employed'].describe()

count     19351.000000
mean      66914.728907
std      139030.880527
min          24.141633
25%         927.009265
50%        2194.220567
75%        5537.882441
max      401755.400475
Name: days_employed, dtype: float64

Se corrigen valores negativos y el siguiente problema a revisar tiene que ver con los números "grandes". En este caso, se realizará el cálculo para el campo `days_employed` como si fueran horas trabajadas, en vez de días.

In [30]:
# Calcular el equivalente en años, asumiendo que un año tiene 8760 horas
data['days_employed'] = data['days_employed'] / 8760

data['days_employed'].describe()

count    19351.000000
mean         7.638668
std         15.871105
min          0.002756
25%          0.105823
50%          0.250482
75%          0.632178
max         45.862489
Name: days_employed, dtype: float64

Con lo calculado, se entiende que el número máximo correspondería a 45.86 años en la columna `days_employed`, esto se deja corregido.

Respecto a lo mencionado para el tipo de datos que tiene la columna, es de tipo float64. Al tratarse de las horas trabajadas, se descarta considerar el dato como int64, es decir, como número entero.

Más adelante se desarrollarán cálculos para los valores ausentes.

A continuación, se continúa revisando la columna `dob_years`

In [31]:
# Se revisa resumen de columna`dob_years`
data['dob_years'].describe()

count    21525.000000
mean        43.293380
std         12.574584
min          0.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

Claramente hay un error en los datos, ya que no puede haber clientes con edad 0 años, que es el valor mínimo ingresado.
Se revisará los valores atípicos en esta columna para determinar si se reemplazan los valores 0 con la media o mediana.

In [32]:
#'dob_years'

# Se calcula la media y la mediana
media = data['dob_years'].mean()
mediana = data['dob_years'].median()

# Se calcula la diferencia entre la media y la mediana
diferencia_media_mediana = media - mediana
print("Diferencia entre la media y la mediana:", diferencia_media_mediana)

# Calculando el IQR
q1 = data['dob_years'].quantile(0.25)
q3 = data['dob_years'].quantile(0.75)
iqr = q3 - q1

# Identificar valores atípicos potenciales
limite_inferior = q1 - 1.5 * iqr
limite_superior = q3 + 1.5 * iqr
valores_atipicos_dob_years = data[(data['dob_years'] < limite_inferior) | (data['dob_years'] > limite_superior)]['dob_years']
print("Cantidad de valores atípicos potenciales:",valores_atipicos_dob_years.count())

Diferencia entre la media y la mediana: 1.2933797909407687
Cantidad de valores atípicos potenciales: 101


Ahora se calculará el porcentaje:

In [33]:
porcentaje = valores_atipicos_dob_years.count() / len(data)
print(f'El porcentaje de valores atipicos de la columna "dob_years" es de un {porcentaje:.2%}')

El porcentaje de valores atipicos de la columna "dob_years" es de un 0.47%


In [34]:
media = data['dob_years'].mean()
mediana = data['dob_years'].median()

In [35]:
media

43.29337979094077

In [36]:
mediana

42.0

Se tiene un 0.47% de valores atípicos en la columna `dob_years`. Al revisar los valores de la media y mediana, nos podemos percatar que la mediana es un valor entero (42) y la media no (43.3). Por lo que, a pesar del valor de los datos atípicos, se escoge la mediana para reemplazar los valores que están en cero, ya que `dob_years` es por naturaleza un número entero.

In [37]:
data['dob_years'] = data['dob_years'].replace(0,data['dob_years'].median())

In [38]:
# Se revisa resumen de columna`dob_years` para comprobar lo realizado
data['dob_years'].describe()

count    21525.000000
mean        43.490453
std         12.218595
min         19.000000
25%         34.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

Se corrigieron valores 0 en columna `dob_years` y ahora se ve que la edad mínima de los clientes del banco es 19 años.

Ahora se revisará la columna `family_status`

In [39]:
# Se revisan los valores únicos de la columna
print(data['family_status'].unique())

['married' 'civil partnership' 'widow / widower' 'divorced' 'unmarried']


Lo único que se actualizará en esta columna, es el caracter especial en la categoría ``widow / widower`` por un ``_`` para unir los dos términos.

In [40]:
# Reemplazar datos 'widow / widower' por valor 'widow_widower'
data['family_status'] = data['family_status'].replace('widow / widower','widow_widower')

In [41]:
# Se revisan nuevamente los valores únicos de la columna
print(data['family_status'].unique())

['married' 'civil partnership' 'widow_widower' 'divorced' 'unmarried']


Luego de esta actualización, no se detectan problemas en esta columna, ya que los nombres están escritos con minúsculas, se revisó que no tiene valores ausentes, por lo que no hay más registros que se deban actualizar.

Ahora se revisará la columna `gender`

In [42]:
#se revisan los valores únicos de la columna
print(data['gender'].unique())

['F' 'M' 'XNA']


In [43]:
#Se necesita saber la cantidad de data registrada
data['gender'].value_counts()

gender
F      14236
M       7288
XNA        1
Name: count, dtype: int64

'XNA' puede deberse al tercer género o a información incorrecta al introducir los datos. En este caso, es un registro. 
Para poder realizar cálculos de acuerdo a grupos de información, se actualizará este valor a "F", ya que es el que tiene mayor cantidad de registros.

In [44]:
# reemplazar dato 'XNA' por valor 'F'
data['gender'] = data['gender'].replace('XNA','F')

In [45]:
#Verificando los valores
data['gender'].value_counts()

gender
F    14237
M     7288
Name: count, dtype: int64

La columna `gender` fue corregida.

Ahora se revisará la data de `income_type`

In [46]:
# Se revisan los valores únicos en la columna
print(data['income_type'].unique())

['employee' 'retiree' 'business' 'civil servant' 'unemployed'
 'entrepreneur' 'student' 'paternity / maternity leave']


En esta columna, lo único que se actualizará, es el caracter especial en la categoría ``paternity / maternity leave`` por un ``_`` para unir los dos términos.

In [47]:
# Reemplazar datos 'paternity / maternity leave' por valor 'paternity_maternity_leave'
data['income_type'] = data['income_type'].replace('paternity / maternity leave','paternity_maternity_leave')

In [48]:
# Se revisan nuevamente los valores únicos de la columna
print(data['income_type'].unique())

['employee' 'retiree' 'business' 'civil servant' 'unemployed'
 'entrepreneur' 'student' 'paternity_maternity_leave']


Luego de esta actualización, no se detectan más problemas en esta columna, ya que los nombres están escritos con minúsculas, se revisó que no tiene valores ausentes, por lo que no hay más registros que se deban actualizar.

A continuación se realizará corrección de columna `purpose`. Repasando, se verán nuevamente los datos únicos:

In [49]:
print(data['purpose'].unique())

['purchase of the house' 'car purchase' 'supplementary education'
 'to have a wedding' 'housing transactions' 'education' 'having a wedding'
 'purchase of the house for my family' 'buy real estate'
 'buy commercial real estate' 'buy residential real estate'
 'construction of own property' 'property' 'building a property'
 'buying a second-hand car' 'buying my own car'
 'transactions with commercial real estate' 'building a real estate'
 'housing' 'transactions with my real estate' 'cars' 'to become educated'
 'second-hand car purchase' 'getting an education' 'car'
 'wedding ceremony' 'to get a supplementary education'
 'purchase of my own house' 'real estate transactions'
 'getting higher education' 'to own a car' 'purchase of a car'
 'profile education' 'university education'
 'buying property for renting out' 'to buy a car' 'housing renovation'
 'going to university']


Se realizarán las agrupaciones para corregir la data:

In [50]:
data['purpose'] = data['purpose'].replace('purchase of the house','house')
data['purpose'] = data['purpose'].replace('purchase of the house for my family','house')
data['purpose'] = data['purpose'].replace('housing','house')
data['purpose'] = data['purpose'].replace('purchase of my own house','house')
data['purpose'] = data['purpose'].replace('housing transactions','house')
data['purpose'] = data['purpose'].replace('housing renovation','house')
data['purpose'] = data['purpose'].replace('car purchase', 'car')
data['purpose'] = data['purpose'].replace('buying a second-hand car', 'car')
data['purpose'] = data['purpose'].replace('buying my own car', 'car')
data['purpose'] = data['purpose'].replace('cars', 'car')
data['purpose'] = data['purpose'].replace('second-hand car purchase', 'car')
data['purpose'] = data['purpose'].replace('to own a car', 'car')
data['purpose'] = data['purpose'].replace('purchase of a car', 'car')
data['purpose'] = data['purpose'].replace('to buy a car', 'car')
data['purpose'] = data['purpose'].replace('supplementary education','education')
data['purpose'] = data['purpose'].replace('to become educated','education')
data['purpose'] = data['purpose'].replace('getting an education','education')
data['purpose'] = data['purpose'].replace('to get a supplementary education','education')
data['purpose'] = data['purpose'].replace('getting higher education','education')
data['purpose'] = data['purpose'].replace('profile education','education')
data['purpose'] = data['purpose'].replace('university education','education')
data['purpose'] = data['purpose'].replace('going to university','education')
data['purpose'] = data['purpose'].replace('to have a wedding','wedding')
data['purpose'] = data['purpose'].replace('having a wedding','wedding')
data['purpose'] = data['purpose'].replace('wedding ceremony','wedding')
data['purpose'] = data['purpose'].replace('buy real estate','real estate')
data['purpose'] = data['purpose'].replace('buy commercial real estate','real estate')
data['purpose'] = data['purpose'].replace('buy residential real estate','real estate')
data['purpose'] = data['purpose'].replace('transactions with commercial real estate','real estate')
data['purpose'] = data['purpose'].replace('building a real estate','real estate')
data['purpose'] = data['purpose'].replace('transactions with my real estate','real estate')
data['purpose'] = data['purpose'].replace('real estate transactions','real estate')
data['purpose'] = data['purpose'].replace('buying property for renting out','real estate')
data['purpose'] = data['purpose'].replace('construction of own property','property')
data['purpose'] = data['purpose'].replace('building a property','property')

In [51]:
print(data['purpose'].unique())

['house' 'car' 'education' 'wedding' 'real estate' 'property']


Se concluye la reagrupación de columna `purpose`

A continuación se revisará si existe data duplicada:

In [52]:
# Identificar las filas duplicadas
filas_duplicadas = data.duplicated(keep=False)

filas_duplicadas_data = data[filas_duplicadas]

print('Hay ',filas_duplicadas_data['gender'].count(),'filas duplicadas\n\n')

#mostrar primeras 5 filas duplicadas
filas_duplicadas_data.head()

Hay  496 filas duplicadas




,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,tiene_hijos
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,real estate,0
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,car,0
67,0,NaN,52,bachelor's degree,0,married,0,F,retiree,0,NaN,house,0
90,2,NaN,35,bachelor's degree,0,married,0,F,employee,0,NaN,house,1
97,0,NaN,47,bachelor's degree,0,married,0,F,employee,0,NaN,education,0


Según el resultado, hay 496 filas duplicadas. Una fila duplicada es la número 120, se va a buscar el duplicado para este caso:

In [53]:
# Se obtiene la fila a buscar (fila 120 en este caso)
fila_a_buscar = data.loc[120]
# Identificar la fila duplicada que tiene los mismos valores que la fila a buscar
fila_duplicada_data_120 = data[data.apply(lambda row: row.equals(fila_a_buscar), axis=1)]

fila_duplicada_data_120

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,tiene_hijos
120,0,NaN,46,secondary education,1,married,0,F,employee,0,NaN,education,0
16378,0,NaN,46,secondary education,1,married,0,F,employee,0,NaN,education,0
16602,0,NaN,46,secondary education,1,married,0,F,employee,0,NaN,education,0
18190,0,NaN,46,secondary education,1,married,0,F,employee,0,NaN,education,0


Aparentemente son datos duplicados, pero no se tiene certeza que sea el mismo cliente, por lo que no se puede asumir que sea un dato duplicado y lo que realmente haría sería solicitar la data con los ID de cliente para determinar si son duplicados o no.
Para este caso se asume que no se recibirá esa data, por lo tanto, en esa situación se calculará el porcentaje de duplicados respecto del total.

In [54]:
porcentaje = filas_duplicadas_data['education'].count() / len(data)
print(f'El porcentaje de filas duplicadas es de un: {porcentaje:.2%}')

El porcentaje de filas duplicadas es de un: 2.30%


Es un porcentaje menor, por lo que se eliminarán las filas con datos duplicados.

In [55]:
# Eliminar las filas duplicadas (incluyendo filas con valores NaN) del DataFrame
data = data.drop_duplicates(keep=False)

In [56]:
# Última comprobación para ver si tenemos duplicados
# Identificar las filas duplicadas
filas_duplicadas = data.duplicated(keep=False)

filas_duplicadas_data = data[filas_duplicadas]

print(filas_duplicadas_data)

Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose, tiene_hijos]
Index: []


In [57]:
# Se obtiene el tamaño del conjunto de datos
num_filas, num_columnas = data.shape

print(f'El conjunto de datos tiene {num_filas} filas y {num_columnas} columnas.')

El conjunto de datos tiene 21029 filas y 13 columnas.


El conjunto de datos ahora está sin las 496 filas duplicadas, quedando con un total de 21029 filas.

## Trabajar con valores ausentes

Se necesita tener el diccionario de family_status con family_status_id, ya que la pregunta inicial es si el estado civil del cliente tiene impacto en el pago de un préstamo.

In [58]:
diccionario_family_status = {
    'married': 0,
    'civil partnership': 1,
    'widow_widower': 2,
    'divorced': 3,
    'unmarried': 4
}

data['family_status_index'] = data['family_status'].map(diccionario_family_status)

data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,tiene_hijos,family_status_index
0,1,0.963205,42,bachelor's degree,0,married,0,F,employee,0,40620.102,house,1,0
1,1,0.459452,36,secondary education,1,married,0,F,employee,0,17932.802,car,1,0
2,0,0.641943,33,secondary education,1,married,0,M,employee,0,23341.752,house,0,0
3,3,0.470862,32,secondary education,1,married,0,M,employee,0,42820.568,education,1,0
4,0,38.843159,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,wedding,0,1


Otro diccionario que se necesita es el de education con education_id, ya que el nivel educativo de las personas puede incidir en ingreso que percibe. Y esto también tiene relación con el pago de un préstamo.

In [59]:
diccionario_education_index = {
    "bachelor's degree": 0,
    'secondary education': 1,
    'some college': 2,
    'primary education': 3,
    'graduate degree': 4
}

data['education_index'] = data['education'].map(diccionario_education_index)

data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,tiene_hijos,family_status_index,education_index
0,1,0.963205,42,bachelor's degree,0,married,0,F,employee,0,40620.102,house,1,0,0
1,1,0.459452,36,secondary education,1,married,0,F,employee,0,17932.802,car,1,0,1
2,0,0.641943,33,secondary education,1,married,0,M,employee,0,23341.752,house,0,0,1
3,3,0.470862,32,secondary education,1,married,0,M,employee,0,42820.568,education,1,0,1
4,0,38.843159,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,wedding,0,1,1


### Restaurar valores ausentes en `total_income`

La columna `total_income` tiene valores ausentes, para poder arreglarlo, se necesita crear una tabla dinámica, en donde se muestre el ingreso mensual por rango de edades. Se evaluará esto junto con otras columnas que pueden incidir en el pago de los préstamos por parte de los clientes.

Para comenzar, se creará la categoría de edades:

In [60]:
# Función que calcula la categoría de edad
def categoria_edad(edad):
    if edad < 0 or pd.isna(edad):
        return 'NA'
    elif edad <30:
        return '0 a 29 años'
    elif edad <40:
        return '30 a 39 años'
    elif edad <50:
        return '40 a 49 años'
    elif edad <60:
        return '50 a 59 años'
    elif edad <70:
        return '60 a 69 años'
    else:
        return '70+'   

In [61]:
# Verificando la función
age = 25
print(categoria_edad(age))

0 a 29 años


In [62]:
# Crear una nueva columna basada en la función
data['categoria_edad'] = data['dob_years'].apply(categoria_edad)

In [63]:
# Comprobar los valores en la nueva columna
data['categoria_edad'].head(10)

0    40 a 49 años
1    30 a 39 años
2    30 a 39 años
3    30 a 39 años
4    50 a 59 años
5     0 a 29 años
6    40 a 49 años
7    50 a 59 años
8    30 a 39 años
9    40 a 49 años
Name: categoria_edad, dtype: object

El género, el estado civil, la educación, la edad, el tipo de ingreso y si una persona tiene hijos incide en los ingresos de uno. Esto porque:
- El género porque existen brechas salariales, responsabilidades familiares, por dar algunos ejemplos.
- El estado civil, porque puede afectar en oportunidades laborales, la estabilidad financiera, porque las personas casadas pueden tener mayores ingresos debido a la posibilidad de tener dos ingresos y compartir gastos.
- La educación, porque mientras más amplia sea esta, mayores conocimientos adquiere la persona y eso es mejor pagado. También tienen mayores posibilidades de ser ascendidos, lo que significaría mayores ingresos a largo plazo.
- La edad incide porque mientras más joven una persona, puede tener ingresos más bajos por su falta de experiencia laboral.
- El tipo de ingresos tiene relación con la fuente de ingresos, ya que estos pueden ser por trabajos asalariados, por pensiones, emprendimientos o estar cesantes y no tener ingresos. Cada tipo de ingreso varía caso a caso, por lo que afecta la situación financiera.
- El tener hijos puede afectar los ingresos de una persona, ya que podría incurrir en gastos adicionales para el cuidado de los niños. En algunas empresas se discrimina a las madres o padres, lo que incide en los ingresos.

Para continuar, se creará una tabla de datos sin valores ausentes, para poder utilizar los datos para restaurar los valores ausentes.

In [64]:
# Se crea una copia de la tabla original para mantenerla intacta
copia_data = data.copy()

# Preprocesamiento para reemplazar valores ausentes en 'total_income' y 'days_employed' con 0 en tabla copia
copia_data['total_income'] = copia_data['total_income'].fillna(0)
copia_data['days_employed'] = copia_data['days_employed'].fillna(0)

# Se crea tabla con la función de agregación sum.
tabla_data_sum = copia_data.pivot_table(index = 'categoria_edad', 
                              values= 'total_income',
                              columns=['income_type','family_status','children','gender','education'],
                              aggfunc='sum')

tabla_data_sum

income_type             business                                        \
family_status  civil partnership                                         
children                       0                                         
gender                         F                                         
education      bachelor's degree primary education secondary education   
categoria_edad                                                           
0 a 29 años           934969.727               NaN          763441.176   
30 a 39 años         1379909.634         55027.816         1056450.674   
40 a 49 años         1244942.623         11368.245         2030602.547   
50 a 59 años          887545.579               NaN         1202654.509   
60 a 69 años           45773.296               NaN          195596.167   
70+                          NaN               NaN           72640.240   

income_type                                                      \
family_status                                                     
children                                                          
gender                                      M                     
education      some college bachelor's degree primary education   
categoria_edad                                                    
0 a 29 años      423838.943        631013.126         27119.024   
30 a 39 años      75294.629        746145.666               NaN   
40 a 49 años      71629.525       1110228.386               NaN   
50 a 59 años      56432.374        237381.889               NaN   
60 a 69 años            NaN         82748.007               NaN   
70+                     NaN               NaN               NaN   

income_type                                                        \
family_status                                                       
children                                                        1   
gender                                                          F   
education      secondary education some college bachelor's degree   
categoria_edad                                                      
0 a 29 años             774086.660   259848.270        360799.707   
30 a 39 años           1101256.726   123350.199        804782.103   
40 a 49 años           1536511.584    38182.106        414317.435   
50 a 59 años            556513.936          NaN        175363.017   
60 a 69 años             55996.513        0.000               NaN   
70+                            NaN          NaN               NaN   

income_type                       ...           retiree                    \
family_status                     ...     widow_widower                     
children                          ...                 1                     
gender                            ...                 F                     
education      primary education  ... bachelor's degree primary education   
categoria_edad                    ...                                       
0 a 29 años                  NaN  ...               NaN               NaN   
30 a 39 años           27305.901  ...               NaN               NaN   
40 a 49 años           32018.704  ...               NaN               NaN   
50 a 59 años                 NaN  ...          5772.878         26463.164   
60 a 69 años                 NaN  ...         51766.082               NaN   
70+                          NaN  ...               NaN         32186.898   

income_type                                                               \
family_status                                                              
children                                                               2   
gender                                               M                 F   
education      secondary education secondary education primary education   
categoria_edad                                                             
0 a 29 años                    NaN                 NaN               NaN   
30 a 39 años        

In [65]:
# Se crea tabla con valores medios de los ingresos en función de los factores identificados
# Preprocesamiento para reemplazar valores ausentes en 'total_income' y 'days_employed' con 0
copia_data['total_income'] = copia_data['total_income'].fillna(0)
copia_data['days_employed'] = copia_data['days_employed'].fillna(0)

# Se crea tabla con la función de agregación mean.
tabla_data_mean = copia_data.pivot_table(index = 'categoria_edad', 
                              values= 'total_income',
                              columns=['income_type','family_status','children','gender','education'],
                              aggfunc='mean')

tabla_data_mean

income_type             business                                        \
family_status  civil partnership                                         
children                       0                                         
gender                         F                                         
education      bachelor's degree primary education secondary education   
categoria_edad                                                           
0 a 29 años         25971.381306               NaN        23134.581091   
30 a 39 años        38330.823167      18342.605333        22009.389042   
40 a 49 años        29641.491024      11368.245000        24763.445695   
50 a 59 años        31698.056393               NaN        24053.090180   
60 a 69 años        22886.648000               NaN        27942.309571   
70+                          NaN               NaN        24213.413333   

income_type                                                       \
family_status                                                      
children                                                           
gender                                       M                     
education       some college bachelor's degree primary education   
categoria_edad                                                     
0 a 29 años     26489.933938      31550.656300         27119.024   
30 a 39 años    25098.209667      37307.283300               NaN   
40 a 49 años    23876.508333      58433.072947               NaN   
50 a 59 años    18810.791333      26375.765444               NaN   
60 a 69 años             NaN      27582.669000               NaN   
70+                      NaN               NaN               NaN   

income_type                                                        \
family_status                                                       
children                                                        1   
gender                                                          F   
education      secondary education some college bachelor's degree   
categoria_edad                                                      
0 a 29 años           26692.643448  28872.03000      24053.313800   
30 a 39 años          30590.464611  30837.54975      27751.107000   
40 a 49 años          34144.701867  19091.05300      25894.839687   
50 a 59 años          26500.663619          NaN      43840.754250   
60 a 69 años          27998.256500      0.00000               NaN   
70+                            NaN          NaN               NaN   

income_type                       ...           retiree                    \
family_status                     ...     widow_widower                     
children                          ...                 1                     
gender                            ...                 F                     
education      primary education  ... bachelor's degree primary education   
categoria_edad                    ...                                       
0 a 29 años                  NaN  ...               NaN               NaN   
30 a 39 años           27305.901  ...               NaN               NaN   
40 a 49 años           32018.704  ...               NaN               NaN   
50 a 59 años                 NaN  ...          2886.439         26463.164   
60 a 69 años                 NaN  ...         25883.041               NaN   
70+                          NaN  ...               NaN         32186.898   

income_type                                                               \
family_status                                                              
children                                                               2   
gender                                               M                 F   
education      secondary education secondary education primary education   
categoria_edad                                                             
0 a 29 años                    NaN                 NaN               NaN   
30 a 39 

Se buscarán el top 5 de valores mínimos y máximos de la tabla que usa los valores de media:

In [66]:
# Convertir la tabla en un DataFrame 1D utilizando melt
tabla_data_mean_flat = tabla_data_mean.reset_index().melt(id_vars='categoria_edad', var_name=['income_type','family_status','children','gender','education'], value_name='income_mean')

# Obtiene los 5 valores más altos y sus filas y columnas
top_5_valores_mas_altos = tabla_data_mean_flat.nlargest(5, 'income_mean')
top_5_valores_mas_altos[['categoria_edad', 'income_type', 'family_status','children','gender','education','income_mean']].reset_index(drop=True, inplace=True)

# Reemplazar los valores cero por NaN en la columna 'income_mean'
tabla_data_mean_flat['income_mean'].replace(0, np.nan, inplace=True)
# Excluir los valores NaN antes de obtener los 5 valores más bajos
tabla_data_mean_flat_sin_nan = tabla_data_mean_flat.dropna(subset=['income_mean'])
# Obtener los 5 valores más bajos y sus filas y columnas
top_5_valores_mas_bajos = tabla_data_mean_flat_sin_nan.nsmallest(5, 'income_mean')
# Seleccionar las columnas deseadas y restablecer el índice
top_5_valores_mas_bajos = top_5_valores_mas_bajos[['categoria_edad', 'income_type', 'family_status', 'children', 'gender', 'education', 'income_mean']]

In [67]:
top_5_valores_mas_altos

,categoria_edad,income_type,family_status,children,gender,education,income_mean
220,60 a 69 años,business,divorced,1,M,bachelor's degree,216039.297000
74,40 a 49 años,business,civil partnership,1,M,bachelor's degree,151046.555000
2108,40 a 49 años,retiree,civil partnership,1,M,bachelor's degree,99944.952000
198,0 a 29 años,business,divorced,1,F,bachelor's degree,91723.092000
1653,50 a 59 años,employee,married,2,M,bachelor's degree,88778.481667


In [68]:
top_5_valores_mas_bajos

,categoria_edad,income_type,family_status,children,gender,education,income_mean
2457,50 a 59 años,retiree,widow_widower,1,F,bachelor's degree,2886.439
2097,50 a 59 años,retiree,civil partnership,1,F,primary education,4049.374
2183,70+,retiree,divorced,1,F,secondary education,5478.583
2084,40 a 49 años,retiree,civil partnership,0,M,some college,5514.581
2468,40 a 49 años,retiree,widow_widower,1,F,secondary education,5768.392


In [69]:
# Se crea tabla con valores de mediana de los ingresos en función de los factores identificados
# Preprocesamiento para reemplazar valores ausentes en 'total_income' y 'days_employed' con 0
copia_data['total_income'] = copia_data['total_income'].fillna(0)
copia_data['days_employed'] = copia_data['days_employed'].fillna(0)

# Se crea tabla con la función de agregación median
tabla_data_median = copia_data.pivot_table(index = 'categoria_edad', 
                              values= 'total_income',
                              columns=['income_type','family_status','children','gender','education'],
                              aggfunc='median')

tabla_data_median

income_type             business                                        \
family_status  civil partnership                                         
children                       0                                         
gender                         F                                         
education      bachelor's degree primary education secondary education   
categoria_edad                                                           
0 a 29 años           26112.0995               NaN          19259.4200   
30 a 39 años          31705.4685         14116.453          23837.0145   
40 a 49 años          29135.9640         11368.245          22734.2905   
50 a 59 años          29349.7140               NaN          21525.9475   
60 a 69 años          22886.6480               NaN          29617.9140   
70+                          NaN               NaN          24259.6870   

income_type                                                      \
family_status                                                     
children                                                          
gender                                      M                     
education      some college bachelor's degree primary education   
categoria_edad                                                    
0 a 29 años       23854.911        28788.4305         27119.024   
30 a 39 años      23968.488        32976.6025               NaN   
40 a 49 años      23095.920        47108.2810               NaN   
50 a 59 años      25351.110        30144.4640               NaN   
60 a 69 años            NaN        25192.1280               NaN   
70+                     NaN               NaN               NaN   

income_type                                                        \
family_status                                                       
children                                                        1   
gender                                                          F   
education      secondary education some college bachelor's degree   
categoria_edad                                                      
0 a 29 años             25880.7310   26768.9280        23757.8510   
30 a 39 años            30084.0315   26839.3035        22807.9670   
40 a 49 años            27594.6410   19091.0530        26647.9690   
50 a 59 años            25208.5050          NaN        41829.6475   
60 a 69 años            27998.2565       0.0000               NaN   
70+                            NaN          NaN               NaN   

income_type                       ...           retiree                    \
family_status                     ...     widow_widower                     
children                          ...                 1                     
gender                            ...                 F                     
education      primary education  ... bachelor's degree primary education   
categoria_edad                    ...                                       
0 a 29 años                  NaN  ...               NaN               NaN   
30 a 39 años           27305.901  ...               NaN               NaN   
40 a 49 años           32018.704  ...               NaN               NaN   
50 a 59 años                 NaN  ...          2886.439         26463.164   
60 a 69 años                 NaN  ...         25883.041               NaN   
70+                          NaN  ...               NaN         32186.898   

income_type                                                               \
family_status                                                              
children                                                               2   
gender                                               M                 F   
education      secondary education secondary education primary education   
categoria_edad                                                             
0 a 29 años                    NaN                 NaN               NaN   
30 a 39 años        

Se buscarán el top 5 de valores mínimos y máximos de la tabla que usa los valores de mediana:



In [70]:
# Convertir la tabla en un DataFrame 1D utilizando melt
tabla_data_median_flat = tabla_data_median.reset_index().melt(id_vars='categoria_edad', var_name=['income_type','family_status','children','gender','education'], value_name='income_median')

# Obtiene los 5 valores más altos y sus filas y columnas
top_5_valores_mas_altos_median = tabla_data_median_flat.nlargest(5, 'income_median')

# # Obtiene los 5 valores más bajos y sus filas y columnas
# top_5_valores_mas_bajos_median = tabla_data_median_flat.nsmallest(5, 'income_median')



# Reemplazar los valores cero por NaN en la columna 'income_median'
tabla_data_median_flat['income_median'].replace(0, np.nan, inplace=True)
# Excluir los valores NaN antes de obtener los 5 valores más bajos
tabla_data_median_flat_sin_nan = tabla_data_median_flat.dropna(subset=['income_median'])
# Obtener los 5 valores más bajos y sus filas y columnas
top_5_valores_mas_bajos_median = tabla_data_median_flat_sin_nan.nsmallest(5, 'income_median')
# Seleccionar las columnas deseadas y restablecer el índice
top_5_valores_mas_bajos_median = top_5_valores_mas_bajos_median[['categoria_edad', 'income_type', 'family_status', 'children', 'gender', 'education', 'income_median']]

In [71]:
top_5_valores_mas_altos_median

,categoria_edad,income_type,family_status,children,gender,education,income_median
220,60 a 69 años,business,divorced,1,M,bachelor's degree,216039.297
74,40 a 49 años,business,civil partnership,1,M,bachelor's degree,151046.555
2108,40 a 49 años,retiree,civil partnership,1,M,bachelor's degree,99944.952
198,0 a 29 años,business,divorced,1,F,bachelor's degree,91723.092
1239,50 a 59 años,employee,civil partnership,1,M,some college,85636.260


In [72]:
top_5_valores_mas_bajos_median

,categoria_edad,income_type,family_status,children,gender,education,income_median
2457,50 a 59 años,retiree,widow_widower,1,F,bachelor's degree,2886.439
2097,50 a 59 años,retiree,civil partnership,1,F,primary education,4049.374
2183,70+,retiree,divorced,1,F,secondary education,5478.583
2084,40 a 49 años,retiree,civil partnership,0,M,some college,5514.581
2468,40 a 49 años,retiree,widow_widower,1,F,secondary education,5768.392


Al comparar los 5 valores más altos y bajos obtenidos tanto con la media como con la mediana, se observa que el grupo de edad, pero principalmente el tipo de ingreso y educación entregan una tendencia para los valores de ingresos. En los valores más altos un tercio de las personas tienen negocio y todas tienen título universitario. Mientras que en los valores más bajos todos son retirados y un tercio es mayor de 50 años.

Para decidir qué medida usar, se siguen observando las tablas de el top de los 5 valores más altos y los 5 más bajos. Al observar más detenidamente la de los valores altos, hay un ID diferente para cada tabla:
- ID 1653 en la tabla de media, el cual no aparece en el top 5 de la tabla de mediana.

Al mirar ese dato, se entiende que si no aparece en la tabla del top 5 de medianas, es porque los valores son diferentes. Esto indicaría la presencia de valores atípicos. Es por este motivo, que se utilizará la mediana de cada tipo de ingreso para completar los valores ausentes de la columna `total_income`.

In [73]:
#  Función para completar los valores ausentes de 'total_income' con la mediana por grupo de 'income_type'
def fill_nan_with_median_by_income_type(df, column_to_fill, column_to_groupby):
    # Se calcula la mediana de 'column_to_fill' para cada grupo definido por 'column_to_groupby'
    median_by_group = df.groupby(column_to_groupby)[column_to_fill].median()
    
    # Recorre cada grupo y reemplaza los valores ausentes en 'column_to_fill' con la mediana correspondiente
    for group, median_value in median_by_group.items():
        mask = (df[column_to_groupby] == group) & (df[column_to_fill].isnull())
        df.loc[mask, column_to_fill] = median_value

In [74]:
#ejemplo de fila con 'total_income' con nan
fila_vacia = data.loc[12]

In [75]:
fila_vacia['total_income']

nan

In [76]:
# Aplicado a cada fila
data_sin_nan_total_income = fill_nan_with_median_by_income_type(data,'total_income','income_type')
data_sin_nan_total_income

In [77]:
fila_vacia = data.loc[12]
fila_vacia['total_income']

18962.318

In [78]:
# Se comprueba si hay algún error
filas_con_nans_total_income = data[data['total_income'].isna()]
# Se obtiene la cantidad de filas que cumplen la condición
filas_con_nans_total_income = len(filas_con_nans_total_income)
# Se muestra el resultado
print("Cantidad de filas con valores NaN en las columnas 'total_income':", filas_con_nans_total_income)

Cantidad de filas con valores NaN en las columnas 'total_income': 0


Ahora se comprueba que `total_income` tiene 21029 registros al igual que el resto de columnas sin valores ausentes.

In [79]:
# Comprobar el número de entradas en las columnas
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21029 entries, 0 to 21524
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   children             21029 non-null  int64  
 1   days_employed        19351 non-null  float64
 2   dob_years            21029 non-null  int64  
 3   education            21029 non-null  object 
 4   education_id         21029 non-null  int64  
 5   family_status        21029 non-null  object 
 6   family_status_id     21029 non-null  int64  
 7   gender               21029 non-null  object 
 8   income_type          21029 non-null  object 
 9   debt                 21029 non-null  int64  
 10  total_income         21029 non-null  float64
 11  purpose              21029 non-null  object 
 12  tiene_hijos          21029 non-null  int64  
 13  family_status_index  21029 non-null  int64  
 14  education_index      21029 non-null  int64  
 15  categoria_edad       21029 non-null  obje

###  Restaurar valores en `days_employed`

Anteriormente para `days_employed`:
- Se realizó el cálculo y se determina que el número corresponde a las horas trabajadas, las cuales fueron transformadas al número de años trabajados.Por lo tanto, ahora nos referiremos a la cantidad de años trabajados.
- Se corrigieron los números negativos.

La columna `days_employed` tiene valores ausentes, para poder arreglarlo (al igual que para `total_income`) se necesita crear una tabla dinámica, en donde se muestren los años trabajados (`days_employed` corregido) por rango de edades. Para esto, se reutilizará la categoría de edades creada anteriormente (`categoria_edad`).

La edad (usando la categoría de edades), la educación, el tipo de ingreso, el estado civil y si una persona tiene hijos incide en los años trabajados. Esto porque:

* La edad de una persona puede estar relacionada con la cantidad de años que ha trabajado. Es probable que las personas mayores tengan una mayor cantidad de años trabajados en comparación con las personas más jóvenes.
* El nivel educativo de una persona puede influir en su historial laboral y la duración de su empleo. Aquellas personas con niveles educativos más altos, pueden tener una mayor experiencia laboral y, por lo tanto, más años trabajados.
* El tipo de ingreso puede estar relacionado con la estabilidad y duración del empleo. Por ejemplo, los empleados a tiempo completo pueden tener una mayor cantidad de años trabajados en comparación con personas que trabajan a tiempo parcial o con contratos temporales.
* El estado civil de una persona también puede influir en su historial laboral. Por ejemplo, las personas casadas o con hijos pueden tener una mayor estabilidad laboral, por ende, tener mayor cantidad de años trabajados.
* El número de hijos puede estar relacionado con la cantidad de años que una persona ha trabajado. Aquellas personas con más hijos pueden haber tenido carreras laborales más largas para mantener a su familia.

Para continuar, se creará una tabla de datos sin valores ausentes, para poder utilizar los datos para restaurar los valores ausentes.

In [80]:
# Se crea una copia de la tabla original para mantenerla intacta
copia_data_dos = data.copy()

# Preprocesamiento para reemplazar valores ausentes en 'days_employed' con 0 en tabla copia
copia_data_dos['days_employed'] = copia_data_dos['days_employed'].fillna(0)

# Se crea tabla con la función de agregación sum.
tabla_data_suma = copia_data_dos.pivot_table(index = 'categoria_edad', 
                              values= 'days_employed',
                              columns=['education','income_type','family_status','children'],
                              aggfunc='sum')

tabla_data_suma

education      bachelor's degree                                          \
income_type             business                                           
family_status  civil partnership                                divorced   
children                       0         1         2         3         0   
categoria_edad                                                             
0 a 29 años             5.016346  2.962102  1.369426       NaN  0.242575   
30 a 39 años           10.339642  7.639572  2.512199  0.186496  3.327001   
40 a 49 años           15.848456  3.645228  0.980432  1.047510  5.267525   
50 a 59 años           12.137368  2.244005  0.218762       NaN  5.391705   
60 a 69 años            1.305008       NaN       NaN       NaN  2.630754   
70+                          NaN       NaN       NaN       NaN  0.983506   

education                                                           ...  \
income_type                                                         ...   
family_status                                   married             ...   
children               1         2         3          0          1  ...   
categoria_edad                                                      ...   
0 a 29 años     0.337383       NaN       NaN   9.960764   7.193934  ...   
30 a 39 años    1.376040  0.068548  0.158355  34.050174  22.783195  ...   
40 a 49 años    2.724267  0.194328       NaN  42.591007  18.587119  ...   
50 a 59 años    1.606074       NaN       NaN  29.802150   4.038252  ...   
60 a 69 años    1.165234       NaN       NaN   9.700140   0.164087  ...   
70+                  NaN       NaN       NaN   1.589426        NaN  ...   

education      some college                                              \
income_type        employee                                               
family_status     unmarried                     widow_widower             
children                  0         1         2             0         1   
categoria_edad                                                            
0 a 29 años        4.184442  0.590732  0.047933           NaN       NaN   
30 a 39 años       1.693140  0.254596       NaN           NaN  0.075048   
40 a 49 años       0.311889       NaN  0.121275      0.077779  0.076736   
50 a 59 años       0.618302       NaN       NaN      0.554112       NaN   
60 a 69 años            NaN       NaN       NaN      0.941131       NaN   
70+                     NaN       NaN       NaN           NaN       NaN   

education                                                                       
income_type              retiree                                                
family_status  civil partnership divorced     married  unmarried widow_widower  
children                       0        0           0          0             0  
categoria_edad                                                                  
0 a 29 años                  NaN      NaN         NaN        NaN           NaN  
30 a 39 años                 NaN      NaN         NaN        NaN           NaN  
40 a 49 años           38.294111      NaN   44.811506        NaN     41.426643  
50 a 59 años          164.922216      0.0  340.640728  38.592477     90.318090  
60 a 69 años           44.516347      NaN  334.321491  86.900224    121.961045  
70+                    90.524121      NaN   40.263566        NaN           NaN  

[6 rows x 252 columns]

In [81]:
# Se crea tabla con valores medios de los años trabajados en función de los factores identificados
# Preprocesamiento para reemplazar valores ausentes en 'days_employed' con 0 en tabla copia
copia_data_dos['days_employed'] = copia_data_dos['days_employed'].fillna(0)

# Se crea tabla con la función de agregación sum.
tabla_data_media = copia_data_dos.pivot_table(index = 'categoria_edad', 
                              values= 'days_employed',
                              columns=['education','income_type','family_status','children'],
                              aggfunc='mean')

tabla_data_media

education      bachelor's degree                                          \
income_type             business                                           
family_status  civil partnership                                divorced   
children                       0         1         2         3         0   
categoria_edad                                                             
0 a 29 años             0.089578  0.148105  0.228238       NaN  0.080858   
30 a 39 años            0.184636  0.190989  0.179443  0.046624  0.207938   
40 a 49 años            0.259811  0.202513  0.122554  0.261877  0.263376   
50 a 59 años            0.328037  0.249334  0.218762       NaN  0.269585   
60 a 69 años            0.261002       NaN       NaN       NaN  0.657689   
70+                          NaN       NaN       NaN       NaN  0.983506   

education                                                         ...  \
income_type                                                       ...   
family_status                                  married            ...   
children               1         2         3         0         1  ...   
categoria_edad                                                    ...   
0 a 29 años     0.168691       NaN       NaN  0.122972  0.133221  ...   
30 a 39 años    0.125095  0.034274  0.158355  0.200295  0.173918  ...   
40 a 49 años    0.272427  0.064776       NaN  0.264540  0.247828  ...   
50 a 59 años    0.267679       NaN       NaN  0.270929  0.201913  ...   
60 a 69 años    1.165234       NaN       NaN  0.388006  0.164087  ...   
70+                  NaN       NaN       NaN  1.589426       NaN  ...   

education      some college                                              \
income_type        employee                                               
family_status     unmarried                     widow_widower             
children                  0         1         2             0         1   
categoria_edad                                                            
0 a 29 años        0.087176  0.073842  0.023967           NaN       NaN   
30 a 39 años       0.099596  0.042433       NaN           NaN  0.075048   
40 a 49 años       0.155945       NaN  0.121275      0.077779  0.076736   
50 a 59 años       0.309151       NaN       NaN      0.277056       NaN   
60 a 69 años            NaN       NaN       NaN      0.941131       NaN   
70+                     NaN       NaN       NaN           NaN       NaN   

education                                                                      
income_type              retiree                                               
family_status  civil partnership divorced    married  unmarried widow_widower  
children                       0        0          0          0             0  
categoria_edad                                                                 
0 a 29 años                  NaN      NaN        NaN        NaN           NaN  
30 a 39 años                 NaN      NaN        NaN        NaN           NaN  
40 a 49 años           38.294111      NaN  44.811506        NaN     41.426643  
50 a 59 años           41.230554      0.0  34.064073  38.592477     45.159045  
60 a 69 años           44.516347      NaN  37.146832  21.725056     40.653682  
70+                    45.262061      NaN  40.263566        NaN           NaN  

[6 rows x 252 columns]

Se buscarán el top 5 de valores mínimos y máximos de la tabla que usa los valores de media:

In [82]:
# Convertir la tabla en un DataFrame 1D utilizando melt
tabla_data_media_flat = tabla_data_media.reset_index().melt(id_vars='categoria_edad', var_name=['education','income_type','family_status','children'], value_name='days_employed_mean')

# Obtiene los 5 valores más altos y sus filas y columnas
top_5_valores_mas_altos_dos = tabla_data_media_flat.nlargest(5, 'days_employed_mean')
top_5_valores_mas_altos_dos[['categoria_edad', 'education','income_type','family_status','children' ,'days_employed_mean']].reset_index(drop=True, inplace=True)

# Reemplazar los valores cero por NaN en la columna 'income_mean'
tabla_data_media_flat['days_employed_mean'].replace(0, np.nan, inplace=True)
# Excluir los valores NaN antes de obtener los 5 valores más bajos
tabla_data_media_flat_sin_nan = tabla_data_media_flat.dropna(subset=['days_employed_mean'])
# Obtener los 5 valores más bajos y sus filas y columnas
top_5_valores_mas_bajos_dos = tabla_data_media_flat_sin_nan.nsmallest(5, 'days_employed_mean')
# Seleccionar las columnas deseadas y restablecer el índice
top_5_valores_mas_bajos_dos = top_5_valores_mas_bajos_dos[['categoria_edad', 'education','income_type','family_status','children', 'days_employed_mean']]

In [83]:
top_5_valores_mas_altos_dos

,categoria_edad,education,income_type,family_status,children,days_employed_mean
723,50 a 59 años,primary education,retiree,widow_widower,1,45.778612
1214,40 a 49 años,secondary education,retiree,married,4,45.594323
687,50 a 59 años,primary education,retiree,divorced,1,45.587913
1487,70+,some college,retiree,civil partnership,0,45.262061
1181,70+,secondary education,retiree,divorced,1,45.212047


In [84]:
top_5_valores_mas_bajos_dos

,categoria_edad,education,income_type,family_status,children,days_employed_mean
1008,0 a 29 años,secondary education,employee,civil partnership,4,0.007919
498,0 a 29 años,primary education,business,divorced,0,0.012360
86,40 a 49 años,bachelor's degree,business,unmarried,2,0.015573
494,40 a 49 años,primary education,business,civil partnership,1,0.017829
1404,0 a 29 años,some college,employee,divorced,0,0.021798


In [85]:
# Se crea tabla con valores de mediana de los años trabajados en función de los factores identificados
# Preprocesamiento para reemplazar valores ausentes en 'days_employed' con 0 en tabla copia
copia_data_dos['days_employed'] = copia_data_dos['days_employed'].fillna(0)

# Se crea tabla con la función de agregación sum.
tabla_data_mediana = copia_data_dos.pivot_table(index = 'categoria_edad', 
                              values= 'days_employed',
                              columns=['education','income_type','family_status','children'],
                              aggfunc='median')

tabla_data_mediana

education      bachelor's degree                                          \
income_type             business                                           
family_status  civil partnership                                divorced   
children                       0         1         2         3         0   
categoria_edad                                                             
0 a 29 años             0.078151  0.135290  0.192005       NaN  0.083330   
30 a 39 años            0.144274  0.141151  0.124774  0.034496  0.156124   
40 a 49 años            0.220454  0.188409  0.066251  0.235789  0.174553   
50 a 59 años            0.205457  0.200889  0.218762       NaN  0.331184   
60 a 69 años            0.268367       NaN       NaN       NaN  0.522320   
70+                          NaN       NaN       NaN       NaN  0.983506   

education                                                         ...  \
income_type                                                       ...   
family_status                                  married            ...   
children               1         2         3         0         1  ...   
categoria_edad                                                    ...   
0 a 29 años     0.168691       NaN       NaN  0.099731  0.093461  ...   
30 a 39 años    0.084779  0.034274  0.158355  0.162459  0.117214  ...   
40 a 49 años    0.201755  0.028128       NaN  0.193595  0.251394  ...   
50 a 59 años    0.213059       NaN       NaN  0.179739  0.149009  ...   
60 a 69 años    1.165234       NaN       NaN  0.282068  0.164087  ...   
70+                  NaN       NaN       NaN  1.589426       NaN  ...   

education      some college                                              \
income_type        employee                                               
family_status     unmarried                     widow_widower             
children                  0         1         2             0         1   
categoria_edad                                                            
0 a 29 años        0.067473  0.092564  0.023967           NaN       NaN   
30 a 39 años       0.090847  0.036770       NaN           NaN  0.075048   
40 a 49 años       0.155945       NaN  0.121275      0.077779  0.076736   
50 a 59 años       0.309151       NaN       NaN      0.277056       NaN   
60 a 69 años            NaN       NaN       NaN      0.941131       NaN   
70+                     NaN       NaN       NaN           NaN       NaN   

education                                                                      
income_type              retiree                                               
family_status  civil partnership divorced    married  unmarried widow_widower  
children                       0        0          0          0             0  
categoria_edad                                                                 
0 a 29 años                  NaN      NaN        NaN        NaN           NaN  
30 a 39 años                 NaN      NaN        NaN        NaN           NaN  
40 a 49 años           38.294111      NaN  44.811506        NaN     41.426643  
50 a 59 años           41.369764      0.0  41.994237  38.592477     45.159045  
60 a 69 años           44.516347      NaN  40.951196  21.423011     40.778865  
70+                    45.262061      NaN  40.263566        NaN           NaN  

[6 rows x 252 columns]

Se buscarán el top 5 de valores mínimos y máximos de la tabla que usa los valores de mediana:

In [86]:
# Convertir la tabla en un DataFrame 1D utilizando melt
tabla_data_mediana_flat = tabla_data_mediana.reset_index().melt(id_vars='categoria_edad', var_name=['education','income_type','family_status','children'], value_name='days_employed_median')

# Obtiene los 5 valores más altos y sus filas y columnas
top_5_valores_mas_altos_mediana_dos = tabla_data_mediana_flat.nlargest(5, 'days_employed_median')
top_5_valores_mas_altos_mediana_dos[['categoria_edad', 'education','income_type','family_status','children' ,'days_employed_median']].reset_index(drop=True, inplace=True)

# Reemplazar los valores cero por NaN en la columna 'income_mean'
tabla_data_mediana_flat['days_employed_median'].replace(0, np.nan, inplace=True)
# Excluir los valores NaN antes de obtener los 5 valores más bajos
tabla_data_mediana_flat_sin_nan = tabla_data_mediana_flat.dropna(subset=['days_employed_median'])
# Obtener los 5 valores más bajos y sus filas y columnas
top_5_valores_mas_bajos_mediana_dos = tabla_data_mediana_flat_sin_nan.nsmallest(5, 'days_employed_median')
# Seleccionar las columnas deseadas y restablecer el índice
top_5_valores_mas_bajos_mediana_dos = top_5_valores_mas_bajos_mediana_dos[['categoria_edad', 'education','income_type','family_status','children', 'days_employed_median']]

In [87]:
top_5_valores_mas_altos_mediana_dos

,categoria_edad,education,income_type,family_status,children,days_employed_median
723,50 a 59 años,primary education,retiree,widow_widower,1,45.778612
1214,40 a 49 años,secondary education,retiree,married,4,45.594323
687,50 a 59 años,primary education,retiree,divorced,1,45.587913
1487,70+,some college,retiree,civil partnership,0,45.262061
1180,60 a 69 años,secondary education,retiree,divorced,1,45.259882


In [88]:
top_5_valores_mas_bajos_mediana_dos

,categoria_edad,education,income_type,family_status,children,days_employed_median
1008,0 a 29 años,secondary education,employee,civil partnership,4,0.007919
498,0 a 29 años,primary education,business,divorced,0,0.012360
86,40 a 49 años,bachelor's degree,business,unmarried,2,0.015573
494,40 a 49 años,primary education,business,civil partnership,1,0.017829
240,0 a 29 años,bachelor's degree,employee,civil partnership,2,0.018429


Al comparar los 5 valores más altos y bajos obtenidos tanto con la media como con la mediana, se cumple con lo que se esperaba, que a mayor edad, las personas iban a tener mayor cantidad de años trabajados. En la tabla con el top 5 de valores más altos, 4 de los 5 registros son mayores de 50 años y en la tabla con el top 5 de valores más bajos, tres de los 5 registros corresponden a personas en el rango de 20 a  29 años. En cuanto a la educación, las personas del grupo de valores más alto tienen en común haber asistido al colegio (tanto escuela primaria, como secundaria), lo cual también podría haber sido esperado, pues antes las personas no tenían tanto acceso como hoy para ir a la Universidad. Respecto al tipo de ingreso, las personas del top 5 más alto se encuentran todas retiradas, este dato no nos indica qué tipo de empleo tuvieron. Pero lo principal reflejado en estos datos, es que a mayor edad las personas tienen más años trabajados. 

Para decidir qué medida usar, se siguen observando las tablas de el top de los 5 valores más altos y los 5 más bajos. Al observar más detenidamente la ambas tablas, hay un ID diferente para cada tabla:
- ID 1181 en la tabla de media, el cual no aparece en el top 5 de la tabla de mediana de valores más altos.
- ID 1404 en la tabla de media, el cual no aparece en el top 5 de la tabla de mediana de valores más bajos.

Al mirar ese dato, se entiende que si no aparece en la tabla del top 5 de medianas, es porque los valores son diferentes. Esto indicaría la presencia de valores atípicos. Es por este motivo, que se utilizará la mediana de cada grupo de edades para completar los valores ausentes de la columna `days_employed`.

In [89]:
#  Función para completar los valores ausentes de 'days_employed' con la mediana por grupo de 'categoria_edad'
def fill_nan_with_median_by_categoria_edad(df, column_to_fill, column_to_groupby):
    # Se calcula la mediana de 'column_to_fill' para cada grupo definido por 'column_to_groupby'
    median_by_group = df.groupby(column_to_groupby)[column_to_fill].median()
    
    # Recorre cada grupo y reemplaza los valores ausentes en 'column_to_fill' con la mediana correspondiente
    for group, median_value in median_by_group.items():
        mask = (df[column_to_groupby] == group) & (df[column_to_fill].isnull())
        df.loc[mask, column_to_fill] = median_value

In [90]:
#ejemplo de fila con 'days_employed' con nan
fila_vacia_days_employed = data.loc[12]

In [91]:
fila_vacia_days_employed['days_employed']

nan

In [92]:
# Aplicado a cada fila
data_sin_nan_days_employed = fill_nan_with_median_by_categoria_edad(data,'days_employed','categoria_edad')
data_sin_nan_days_employed

In [93]:
#se verifica en la fila revisada anteriormente
fila_vacia_days_employed = data.loc[12]
fila_vacia_days_employed['days_employed']

40.51776473665543

In [94]:
# Se comprueba si hay algún error
filas_con_nans_days_employed = data[data['total_income'].isna()]
# Se obtiene la cantidad de filas que cumplen la condición
filas_con_nans_days_employed = len(filas_con_nans_days_employed)
# Se muestra el resultado
print("Cantidad de filas con valores NaN en las columnas 'days_employed':", filas_con_nans_days_employed)

Cantidad de filas con valores NaN en las columnas 'days_employed': 0


Ahora se comprueba que `days_employed` tiene 21029 registros al igual que el resto de columnas sin valores ausentes.

In [95]:
# Comprobar el número de entradas en las columnas
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21029 entries, 0 to 21524
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   children             21029 non-null  int64  
 1   days_employed        21029 non-null  float64
 2   dob_years            21029 non-null  int64  
 3   education            21029 non-null  object 
 4   education_id         21029 non-null  int64  
 5   family_status        21029 non-null  object 
 6   family_status_id     21029 non-null  int64  
 7   gender               21029 non-null  object 
 8   income_type          21029 non-null  object 
 9   debt                 21029 non-null  int64  
 10  total_income         21029 non-null  float64
 11  purpose              21029 non-null  object 
 12  tiene_hijos          21029 non-null  int64  
 13  family_status_index  21029 non-null  int64  
 14  education_index      21029 non-null  int64  
 15  categoria_edad       21029 non-null  obje

## Clasificación de datos

Las preguntas a responder son:

- ¿Hay alguna conexión entre tener hijos y reembolsar un préstamo a tiempo?
- ¿Hay alguna conexión entre el estado civil y el reembolso de un préstamo a tiempo?
- ¿Hay alguna conexión entre el nivel de ingresos y el reembolso de un préstamo a tiempo?
- ¿Cómo afectan los diferentes propósitos de un préstamo a su reembolso a tiempo?

  Para revisar esto, se clasificará la data en orden de lo consultado:

### ¿Hay alguna conexión entre tener hijos y reembolsar un préstamo a tiempo?

Para esto se calculará la tasa promedio de reembolso para personas que tienen hijos (tiene_hijos = 1) y para personas que no tienen hijos (tiene_hijos = 0). Comparando estas tasas, se podrá determinar si hay alguna conexión entre tener hijos y reembolsar un préstamo a tiempo.

In [96]:
#Se calculará la tasa de reembolso promedio para cada grupo (0 = NO, 1 = SI)
tasa_reembolso_por_hijos = data.groupby('tiene_hijos')['debt'].mean()
print(tasa_reembolso_por_hijos)

tiene_hijos
0    0.077164
1    0.093145
Name: debt, dtype: float64


Al ver estos resultados, podemos concluir que alrededor de un 7.71% de las personas que no tienen hijos han reembolsado su préstamo a tiempo, mientras que las personas que tienen hijos tienen una tasa de 9.31%.

### ¿Hay alguna conexión entre el estado civil y el reembolso de un préstamo a tiempo?

In [97]:
# Se calculará la tasa de reembolso promedio para cada grupo (Estado_Civil)
tasa_reembolso_por_estado_civil = data.groupby('family_status_index')['debt'].mean()
print(tasa_reembolso_por_estado_civil)

family_status_index
0    0.077275
1    0.094912
2    0.066667
3    0.071249
4    0.098526
Name: debt, dtype: float64


- La tasa de reembolso promedio para personas que están casadas (family_status_index = 0) es aproximadamente 0.0773, o el 7.73%
- La tasa de reembolso promedio para personas en una unión civil (family_status_index = 1) es aproximadamente 0.0949, o el 9.49%
- La tasa de reembolso promedio para personas que son viudas o viudos (family_status_index = 2) es aproximadamente 0.0667, o el 6.67%
- La tasa de reembolso promedio para personas divorciadas (family_status_index = 3) es aproximadamente 0.0712, o el 7.12%
- La tasa de reembolso promedio para personas solteras o no casadas (family_status_index = 4) es aproximadamente 0.0985, o el 9.85%

### ¿Hay alguna conexión entre el nivel de ingresos y el reembolso de un préstamo a tiempo?

Para analizar esto se realizará cálculo de la correlación entre las dos variables:

In [98]:
# Se calculará la correlación entre 'Ingreso_Mensual' y 'Pago_Tiempo'
correlacion = data['total_income'].corr(data['debt'])

print(f"La correlación entre Ingreso Mensual y Pago a Tiempo: {correlacion}")

La correlación entre Ingreso Mensual y Pago a Tiempo: -0.014148295064713486


### ¿Cómo afectan los diferentes propósitos de un préstamo a su reembolso a tiempo?

In [99]:
# Se calculará la tasa de reembolso promedio para cada propósito de préstamo
tasa_reembolso_por_proposito = data.groupby('purpose')['debt'].mean()

print(tasa_reembolso_por_proposito)

purpose
car            0.094665
education      0.094364
house          0.068633
property       0.073757
real estate    0.077647
wedding        0.081794
Name: debt, dtype: float64


Se puede concluir que:
- Aproximadamente el 9.47% de los préstamos con el propósito de "car" se reembolsan a tiempo.
- Aproximadamente el 9.44% de los préstamos con el propósito de "education" se reembolsan a tiempo.
- Aproximadamente el 6.86% de los préstamos con el propósito de "house" se reembolsan a tiempo.
- Aproximadamente el 7.38% de los préstamos con el propósito de "property" se reembolsan a tiempo.
- Aproximadamente el 7.76% de los préstamos con el propósito de "real estate" se reembolsan a tiempo.
- Aproximadamente el 8.17% de los préstamos con el propósito de "wedding" se reembolsan a tiempo.

## Comprobación de las hipótesis


**¿Existe una correlación entre tener hijos y pagar a tiempo?**

In [100]:
print(tasa_reembolso_por_hijos)

tiene_hijos
0    0.077164
1    0.093145
Name: debt, dtype: float64


**Conclusión**

Al ver los resultados, podemos concluir que alrededor de un 7.71% de las personas que no tienen hijos han reembolsado su préstamo a tiempo, mientras que las personas que tienen hijos tienen una tasa de 9.31%.

Si comparamos las tasas de reembolso promedio entre los dos grupos, podemos observar que el grupo con "tiene_hijos" = 1 (personas que tienen hijos) tiene una tasa de reembolso ligeramente mayor que el grupo con "tiene_hijos" = 0 (personas que no tienen hijos). Sin embargo, la diferencia entre las dos tasas no es significativamente grande en este caso, por lo que no se podría afirmar que haya alguna conexión entre tener hijos y reembolsar un préstamo a tiempo.


**¿Existe una correlación entre la situación familiar y el pago a tiempo?**

In [101]:
print(tasa_reembolso_por_estado_civil)

family_status_index
0    0.077275
1    0.094912
2    0.066667
3    0.071249
4    0.098526
Name: debt, dtype: float64


**Conclusión**

Al ver los resultados se puede decir que:
- La tasa de reembolso promedio para personas que están casadas (family_status_index = 0) es aproximadamente 0.0773, o el 7.73%
- La tasa de reembolso promedio para personas en una unión civil (family_status_index = 1) es aproximadamente 0.0949, o el 9.49%
- La tasa de reembolso promedio para personas que son viudas o viudos (family_status_index = 2) es aproximadamente 0.0667, o el 6.67%
- La tasa de reembolso promedio para personas divorciadas (family_status_index = 3) es aproximadamente 0.0712, o el 7.12%
- La tasa de reembolso promedio para personas solteras o no casadas (family_status_index = 4) es aproximadamente 0.0985, o el 9.85%

Al analizar la tasa promedio de reembolso para diferentes estados civiles, se pueden observar diferencias en las tasas entre los grupos. Por ejemplo, parece que las personas que están "solteras" tienen la tasa de reembolso más alta en comparación con otros grupos, mientras que las personas viudas o viudos tienen la tasa de reembolso más baja, por lo que sí existe una correlación entre la situación familiar y el pago a tiempo.

**¿Existe una correlación entre el nivel de ingresos y el pago a tiempo?**

In [102]:
correlacion = data['total_income'].corr(data['debt'])

print(f"La correlación entre Ingreso Mensual y Pago a Tiempo: {correlacion}")

La correlación entre Ingreso Mensual y Pago a Tiempo: -0.014148295064713486


**Conclusión**

La correlación, al ser cercana a cero, indica que es una correlación muy débil entre estas dos variables en los datos. Una correlación cercana a cero sugiere que no existe una relación lineal fuerte entre el ingreso mensual y la probabilidad de realizar el pago a tiempo de los préstamos.

**¿Cómo afecta el propósito del crédito a la tasa de incumplimiento?**

In [103]:
print(tasa_reembolso_por_proposito)

purpose
car            0.094665
education      0.094364
house          0.068633
property       0.073757
real estate    0.077647
wedding        0.081794
Name: debt, dtype: float64


**Conclusión**

Se puede concluir que:
- Aproximadamente el 9.47% de los préstamos con el propósito de "car" se reembolsan a tiempo.
- Aproximadamente el 9.44% de los préstamos con el propósito de "education" se reembolsan a tiempo.
- Aproximadamente el 6.86% de los préstamos con el propósito de "house" se reembolsan a tiempo.
- Aproximadamente el 7.38% de los préstamos con el propósito de "property" se reembolsan a tiempo.
- Aproximadamente el 7.76% de los préstamos con el propósito de "real estate" se reembolsan a tiempo.
- Aproximadamente el 8.17% de los préstamos con el propósito de "wedding" se reembolsan a tiempo.

Analizando estas tasas de reembolso, se pueden observar las diferencias en el pago a tiempo de los préstamos según el propósito del préstamo. Por ejemplo, parece que los préstamos con el propósito de "car" y "education" tienen tasas de reembolso promedio ligeramente más altas en comparación con los préstamos con los propósitos de "house", "property", "real estate" y "wedding".


# Conclusión general

Se comenzó revisando la data de las diferentes columnas y se realizaron las siguientes correcciones:

* Se corrigieron los datos para la columna `education`, dejando los datos únicos. Para esto se eliminaron los duplicados, escribiendo todo en minúsculas.

* En `children`, se corrigieron los datos negativos y los que indicaban 20 hijos, esto porque estos datos eran menor al 0.5%.
Luego, se creó la columna `tiene_hijos`, para agrupar la data. Esto nos permitió responder a una de las preguntas realizadas inicialmente.

* Para la columna `days_employed` se corrigió lo siguiente:

* Existía un 73.9% de data con número negativo, por lo que se aplicó el valor absoluto para corregir la data, ya que no se podían borrar las filas.
    * Se determinó que los números “grandes” correspondían a  años en la columna, realizando la conversión se corrigió la data
    * El tipo de datos que tiene la columna, es de tipo float64. Al tratarse de las horas trabajadas, se descartó considerar el dato como int64, es decir, como número entero.

Luego se revisaron los valores ausentes: estos se encontraban en las columnas `days_employed` y `total_income`. Se verificó que se encontraban en las mismas filas. Es por esto que se buscó si había un patrón.  Se descartó que la edad (`dob_years`) formara parte de un patrón, porque los valores mínimos eran 0 y 19 años:
* Cero años, debe ser un error de ingreso de datos.
* 19 años, ya es mayor de edad por lo que podría trabajar.

Finalmente, no se encontró un patrón para determinar el comportamiento de valores ausentes. 
Luego de eso, se corrigieron los valores cero en la columna `dob_years`, esto porque no puede haber clientes con edad 0 años, que es el valor mínimo ingresado. Se revisaron los valores atípicos en esta columna para determinar si se reemplazan los valores 0 con la media o mediana. El porcentaje de valores atípicos de la columna es de un 0.47%, sin embargo, al observar los valores de media y mediana se pudo notar que la mediana correspondía a un número entero, razón por la cual se selecciona para reemplazar los valores que estaban en cero.

* Se corrigió la data de `gender`, ya que solamente había un registro 'XNA'. Esto podría deberse al tercer género o a información incorrecta al introducir los datos. Para poder realizar cálculos de acuerdo a grupos de información, se actualizaó este valor a "F", ya que es el que tiene mayor cantidad de registros.

* Se realizó la corrección para la columna `purpose`, agrupando por categoría los motivos, los cuales fueron utilizados para responder una de las preguntas de este estudio.

Para poder responder las preguntas de este estudio, se necesitó crear dos diccionarios: 
* Primero el de `family_status` con `family_status_id`, ya que la pregunta inicial es si el estado civil del cliente tiene impacto en el pago de un préstamo.
* Luego se creó el diccionario de `education` con `education_id`, ya que el nivel educativo de las personas puede incidir en ingreso que percibe. Y esto también tiene relación con el pago de un préstamo.	

Luego se creó la categoría de edades, para poder crear las tablas dinámicas (o pivote) para analizar si se completaban los valores ausentes de `total_income` y `days_employed`

En el caso de `total_income`, los datos que se consideraron como criterio dentro de las tablas dinámicas fueron el género, el estado civil, la educación, la edad y si una persona tiene hijos incide en los ingresos de uno. Esta es la explicación:
* El género porque existen brechas salariales, responsabilidades familiares, por dar algunos ejemplos.
* El estado civil, porque puede afectar en oportunidades laborales, la estabilidad financiera, porque las personas casadas pueden tener mayores ingresos debido a la posibilidad de tener dos ingresos y compartir gastos.
* La educación, porque mientras más amplia sea esta, mayores conocimientos adquiere la persona y eso es mejor pagado. También tienen mayores posibilidades de ser ascendidos, lo que significaría mayores ingresos a largo plazo.
* La edad incide porque mientras más joven una persona, puede tener ingresos más bajos por su falta de experiencia laboral.
* El tipo de ingresos tiene relación con la fuente de ingresos, ya que estos pueden ser por trabajos asalariados, por pensiones, emprendimientos o estar cesantes y no tener ingresos. Cada tipo de ingreso varía caso a caso, por lo que afecta la situación financiera.
* El tener hijos puede afectar los ingresos de una persona, ya que podría incurrir en gastos adicionales para el cuidado de los niños. En algunas empresas se discrimina a las madres o padres, lo que incide en los ingresos.

Para `total_income`, se compararon los top 5 de medias y medianas. Se observó que había un registro que no aparecía en la tabla del top 5 de las medianas. Al mirar ese dato, se entiende que si no aparece en esa tabla, es porque los valores son diferentes. Esto indicaría la presencia de valores atípicos. Es por este motivo, que se utilizó la mediana para completar los valores ausentes de la columna `total_income`. Para calcular la mediana, se creó la función que reemplaza los valores ausentes de ``total_income`` con la mediana por grupo de ``income_type``.

Después se aplicó la misma lógica para `days_employed`. Los datos que se consideraron como criterio dentro de las tablas dinámicas fueron: La edad (usando la categoría de edades antes creada), la educación, el tipo de ingreso, el estado civil y si una persona tiene hijos incide en los años trabajados. Esto porque:

* La edad de una persona puede estar relacionada con la cantidad de años que ha trabajado. Es probable que las personas mayores tengan una mayor cantidad de años trabajados en comparación con las personas más jóvenes.
* El nivel educativo de una persona puede influir en su historial laboral y la duración de su empleo. Aquellas personas con niveles educativos más altos, pueden tener una mayor experiencia laboral y, por lo tanto, más años trabajados.
* El tipo de ingreso puede estar relacionado con la estabilidad y duración del empleo. Por ejemplo, los empleados a tiempo completo pueden tener una mayor cantidad de años trabajados en comparación con personas que trabajan a tiempo parcial o con contratos temporales.
* El estado civil de una persona también puede influir en su historial laboral. Por ejemplo, las personas casadas o con hijos pueden tener una mayor estabilidad laboral, por ende, tener mayor cantidad de años trabajados.
* El número de hijos puede estar relacionado con la cantidad de años que una persona ha trabajado. Aquellas personas con más hijos pueden haber tenido carreras laborales más largas para mantener a su familia.

Para `days_employed`, se compararon los top 5 de medias y medianas. Se observó que había un par de registros que no aparecían en las tabla del top 5 de las medianas. Al mirar esos datos, se entiende que si no aparece en esa tabla, es porque los valores son diferentes. Esto indicaría la presencia de valores atípicos. Es por este motivo, que se utilizó la mediana para completar los valores ausentes de la columna `days_employed`. Para calcular la mediana, se creó la función que reemplaza los valores ausentes de de ``days_employed`` con la mediana por grupo de ``categoria_edad``.

Finalmente se realizó el análisis para las preguntas de este estudio:
* ¿Existe una correlación entre tener hijos y pagar a tiempo?  
Al ver los resultados, podemos concluir que alrededor de un 7.71% de las personas que no tienen hijos han reembolsado su préstamo a tiempo, mientras que las personas que tienen hijos tienen una tasa de 9.31%.
Si comparamos las tasas de reembolso promedio entre los dos grupos, podemos observar que el grupo con "tiene_hijos" = 1 (personas que tienen hijos) tiene una tasa de reembolso ligeramente mayor que el grupo con "tiene_hijos" = 0 (personas que no tienen hijos). Sin embargo, la diferencia entre las dos tasas no es significativamente grande en este caso, por lo que no se podría afirmar que haya alguna conexión entre tener hijos y reembolsar un préstamo a tiempo.

* ¿Existe una correlación entre la situación familiar y el pago a tiempo?
Al ver los resultados se puede decir que:
    * La tasa de reembolso promedio para personas que están casadas (family_status_index = 0) es aproximadamente 0.0773, o el 7.73%
    * La tasa de reembolso promedio para personas en una unión civil (family_status_index = 1) es aproximadamente 0.0949, o el 9.49%
    * La tasa de reembolso promedio para personas que son viudas o viudos (family_status_index = 2) es aproximadamente 0.0667, o el 6.67%
    * La tasa de reembolso promedio para personas divorciadas (family_status_index = 3) es aproximadamente 0.0712, o el 7.12%
    * La tasa de reembolso promedio para personas solteras o no casadas (family_status_index = 4) es aproximadamente 0.0985, o el 9.85%
Al analizar la tasa promedio de reembolso para diferentes estados civiles, se pueden observar diferencias en las tasas entre los grupos. Por ejemplo, parece que las personas que están "solteras" tienen la tasa de reembolso más alta en comparación con otros grupos, mientras que las personas viudas o viudos tienen la tasa de reembolso más baja, por lo que sí existe una correlación entre la situación familiar y el pago a tiempo.

* ¿Existe una correlación entre el nivel de ingresos y el pago a tiempo?
La correlación, al ser cercana a cero, indica que es una correlación muy débil entre estas dos variables en los datos. Una correlación cercana a cero sugiere que no existe una relación lineal fuerte entre el ingreso mensual y la probabilidad de realizar el pago a tiempo de los préstamos.

* ¿Cómo afecta el propósito del crédito a la tasa de incumplimiento?
Se puede concluir que:
    * Aproximadamente el 9.47% de los préstamos con el propósito de "car" se reembolsan a tiempo.
    * Aproximadamente el 9.44% de los préstamos con el propósito de "education" se reembolsan a tiempo.
    * Aproximadamente el 6.86% de los préstamos con el propósito de "house" se reembolsan a tiempo.
    * Aproximadamente el 7.38% de los préstamos con el propósito de "property" se reembolsan a tiempo.
    * Aproximadamente el 7.76% de los préstamos con el propósito de "real estate" se reembolsan a tiempo.
    * Aproximadamente el 8.17% de los préstamos con el propósito de "wedding" se reembolsan a tiempo.
Analizando estas tasas de reembolso, se pueden observar las diferencias en el pago a tiempo de los préstamos según el propósito del préstamo. Por ejemplo, parece que los préstamos con el propósito de "car" y "education" tienen tasas de reembolso promedio ligeramente más altas en comparación con los préstamos con los propósitos de "house", "property", "real estate" y "wedding".